## Demo: BasicActivity
The basic steps to set up an OpenCLSim simulation are:
* Import libraries
* Initialise simpy environment
* Define object classes
* Create objects
  * Create sites
  * Create vessels
  * Create activities
* Register processes and run simpy

----

This notebook shows a very simple OpenCLSim example. Basically we use OpenCLSim to yield a timeout only. Some basic functionality of the logging is shown. 

#### 0. Import libraries

In [1]:
import datetime, time
import simpy

import pandas as pd

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

#### 1. Initialise simpy environment

In [2]:
# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

#### 2. Define object classes
In this simple example we won't define specific classes. We will only use a built in activity from OpenCLSim.

#### 3. Create objects
##### 3.1. Create site object(s)
No site objects are created.

##### 3.2. Create vessel object(s)
No vessel objects are created.

##### 3.3 Create activity/activities

In [3]:
# initialise registry
registry = {}

In [4]:
# create a basic activity (it just creates an event that shifts time for 'duration')
activity = model.BasicActivity(
    env=my_env,
    name="Basic activity",
    registry=registry,
    duration=42,
)

#### 4. Register processes and run simpy

In [5]:
# initate the simpy processes defined in the 'basic activity' and run simpy
model.register_processes([activity])
my_env.run()

#### 5. Inspect results
##### 5.1 Inspect logs
The method plot.get_log_dataframe returns the log of an activity in the form of a dataframe. By adding other activities in a list as the second argument, the Activity can be made more human readable. 

In [6]:
plot.get_log_dataframe(activity)

,Activity,Timestamp,ActivityState
0,bc355df0-b2db-4dbd-9e36-37fc31cc8494,1970-01-01 00:00:00,START
1,bc355df0-b2db-4dbd-9e36-37fc31cc8494,1970-01-01 00:00:42,STOP


In [7]:
plot.get_log_dataframe(activity, [activity])

,Activity,Timestamp,ActivityState
0,Basic activity,1970-01-01 00:00:00,START
1,Basic activity,1970-01-01 00:00:42,STOP


#### 6. Additional logging
The scenario can be extended by adding additional logging instances, that is, the information of the basic activity will also be logged in the activity log of additional activities. This is useful for more complex nested activities. In this example a reporting activity is added where the basic activity log is also added.

In [8]:
# initiate SimPy environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

# create activities
registry = {}
reporting_activity = model.BasicActivity(
    env=my_env,
    name="Reporting activity",
    registry=registry,
    duration=0,
)
basic_activity = model.BasicActivity(
    env=my_env,
    name="Basic activity",
    registry=registry,
    duration=42,
    additional_logs=[reporting_activity],
)

# initate the simpy processes defined in the 'while activity' and run simpy
model.register_processes([basic_activity])
my_env.run()

#### 6.1 Inspect results
We now show the dataframe of the reporting_activity log, and use the basic_activity to make it more human readable.

In [9]:
plot.get_log_dataframe(reporting_activity, [basic_activity])

,Activity,Timestamp,ActivityState,type,ref
0,Basic activity,1970-01-01 00:00:00,START,additional log,de553233-2ac6-4085-81aa-cadc21a61431
1,Basic activity,1970-01-01 00:00:42,STOP,additional log,de553233-2ac6-4085-81aa-cadc21a61431
